In [1]:
import pandas as pd
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 1000)
import json
import numpy
from IPython.display import display
from subprocess import check_output
import sys
import traceback
from tqdm.notebook import tqdm, trange
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from paramiko import SSHClient
from scp import SCPClient
from subprocess import call

In [2]:
dat_2 = pd.read_pickle("two_players_data")
dat_3 = pd.read_pickle("three_players_data")
dat_4 = pd.read_pickle("four_players_data")

In [3]:
techs = {
    "tech1": "1q_1o",
    "tech2": "k_per_terra",
    "tech3": "4pip",
    "tech4": "7vp",
    "tech5": "1o_1pw",
    "tech6": "1k_1c",
    "tech7": "3vp_per_gaia_place",
    "tech8": "4c",
    "tech9": "4pw"
}

adv_techs = {
    "advtech1": "3vp_per_fed_pass",
    "advtech2": "2vp_per_tech_bump",
    "advtech3": "1qic_5c_action",
    "advtech4": "2vp_per_mine",
    "advtech5": "3vp_per_rl_pass",
    "advtech6": "1o_per_sector",
    "advtech7": "1vp_per_terra_pass",
    "advtech8": "2vp_per_gaia",
    "advtech9": "4vp_per_ts",
    "advtech10": "2vp_per_sector",
    "advtech11": "3o_action",
    "advtech12": "5vp_per_fed",
    "advtech13": "3k_action",
    "advtech14": "3vp_per_mine_place",
    "advtech15": "3vp_per_ts_place",
}

feds = {
    "fed1": "12vp",
    "fed2": "qic",
    "fed3": "2pw",
    "fed4": "2o",
    "fed5": "6c",
    "fed6": "2k",
    "gleens": "gleens"
}

round_scorings = {
    "score1": "2vp_per_terra",
    "score2": "2vp_per_research_bump",
    "score3": "2vp_per_mine_place",
    "score4": "5vp_per_fed_place",
    "score5": "4vp_per_ts_place",
    "score6": "4vp_per_gaia_place",
    "score7": "5vp_per_3pip_place",
    "score8": "3vp_per_ts_place",
    "score9": "3vp_per_gaia_place",
    "score10": "5vp_per_3pip_place",
}

boosters = {

    "booster1": "1k_1o",
    "booster2": "2pwt_1o",
    "booster3": "1qic_2c",
    "booster4": "2c_terra",
    "booster5": "2pw_nav",
    "booster6": "1o_1vp_per_mine",
    "booster7": "1o_2vp_per_ts",
    "booster8": "1k_3vp_per_rl",
    "booster9": "4pw_4vp_per_3pip",
    "booster10": "4c_1vp_per_gaia",
}

bad_buildings = {"colony", 
                 "colonyShip", 
                 "tradeShip", 
                 "constructionShip", 
                 "researchShip",
                 "scout",
                 "frigate",
                 "battleShip",
                 "customsPost",
                 "tradePost"
                }

factions = {
    "terrans",
    "lantids",
    "hadsch-hallas",
    "ivits",
    "baltaks",
    "geodens",
    "xenos",
    "gleens",
    "ambas",
    "taklons",
    "bescods",
    "firaks",
    "itars",
    "nevlas"
}

final_scorings = {
    "planetType",
    "structure", 
    "sector", 
    "satellite", 
    "gaia",
    "structureFed"
}

In [9]:
def make_final_scoring_heatmap(dat_x, num_players):
    
    final_scorings = [
    "planetType",
    "structure", 
    "sector", 
    "satellite", 
    "gaia",
    "structureFed"
    ]
    
    # clean dat_x
    clean_pos = []
    for pos in range(num_players):
        dat = dat_x[dat_x['pos_' + str(pos + 1) + '_dropped'] == False]
        # dat = dat[dat['pos_' + str(pos + 1) + '_elo'] > 200]
        dat['pos_' + str(pos + 1) + '_score'] += -1 * dat['pos_' + str(pos + 1) + '_score_bid']
        
        clean_pos.append(dat)
    dat_clean = pd.concat(clean_pos, ignore_index=True)
    
    for faction in factions:
        factions_pos = []
        for pos in range(num_players):
            factions_pos.append(dat_clean[dat_clean["pos_" + str(pos + 1) +"_faction"] == faction])
        
        scores = [[None] * len(final_scorings) for i in range(len(final_scorings))]
        scores_overall = [[None] * len(final_scorings) for i in range(len(final_scorings))]
        scores_diff = [[None] * len(final_scorings) for i in range(len(final_scorings))]
        ave_pos = [[None] * len(final_scorings) for i in range(len(final_scorings))]
        tot_pos = [[None] * len(final_scorings) for i in range(len(final_scorings))]
        ave_pos_min_faction = [[None] * len(final_scorings) for i in range(len(final_scorings))]
        
        for pos in range(num_players):
            df = factions_pos[pos]
            dat = clean_pos[pos]
            for i, scoring_1 in enumerate(final_scorings):
                sc1_1 = df[df['final_scoring_1'] == scoring_1]
                sc1_2 = df[df['final_scoring_2'] == scoring_1]
                sc1_1_all = dat[dat['final_scoring_1'] == scoring_1]
                sc1_2_all = dat[dat['final_scoring_2'] == scoring_1]
                for j, scoring_2 in enumerate(final_scorings):
                    if not scoring_1 == scoring_2:
                        sc2_1 = sc1_2[sc1_2['final_scoring_1'] == scoring_2]
                        sc2_2 = sc1_1[sc1_1['final_scoring_2'] == scoring_2]
                        sc2_1_all = sc1_2_all[sc1_2_all['final_scoring_1'] == scoring_2]
                        sc2_2_all = sc1_1_all[sc1_1_all['final_scoring_2'] == scoring_2]
                        
                        if len(sc2_1) > 0:    
                            if scores[i][j] is None:
                                scores[i][j] = sc2_1['pos_' + str(pos + 1) + '_score_final1'].mean()
                                scores[i][j] += sc2_1['pos_' + str(pos + 1) + '_score_final2'].mean()
                            else:
                                scores[i][j] += sc2_1['pos_' + str(pos + 1) + '_score_final1'].mean()
                                scores[i][j] += sc2_1['pos_' + str(pos + 1) + '_score_final2'].mean()
                        if len(sc2_2) > 0:
                            if scores[i][j] is None:
                                scores[i][j] = sc2_2['pos_' + str(pos + 1) + '_score_final1'].mean()
                                scores[i][j] += sc2_2['pos_' + str(pos + 1) + '_score_final2'].mean()
                            else:
                                scores[i][j] += sc2_2['pos_' + str(pos + 1) + '_score_final1'].mean()
                                scores[i][j] += sc2_2['pos_' + str(pos + 1) + '_score_final2'].mean()
                        if len(sc2_1_all) > 0:
                            if scores_overall[i][j] is None:
                                scores_overall[i][j] = sc2_1_all['pos_' + str(pos + 1) + '_score_final1'].mean()
                                scores_overall[i][j] += sc2_1_all['pos_' + str(pos + 1) + '_score_final2'].mean()
                            else:
                                scores_overall[i][j] += sc2_1_all['pos_' + str(pos + 1) + '_score_final1'].mean()
                                scores_overall[i][j] += sc2_1_all['pos_' + str(pos + 1) + '_score_final2'].mean()
                        if len(sc2_2_all) > 0:
                            if scores_overall[i][j] is None:
                                scores_overall[i][j] = sc2_2_all['pos_' + str(pos + 1) + '_score_final1'].mean()
                                scores_overall[i][j] += sc2_2_all['pos_' + str(pos + 1) + '_score_final2'].mean()
                            else:
                                scores_overall[i][j] += sc2_2_all['pos_' + str(pos + 1) + '_score_final1'].mean()
                                scores_overall[i][j] += sc2_2_all['pos_' + str(pos + 1) + '_score_final2'].mean()
                        if pos == 0:
                            ave_pos[i][j] = len(sc2_1) + len(sc2_2)
                        
                        if tot_pos[i][j] is None:
                            tot_pos[i][j] = len(sc2_1) + len(sc2_2)
                        else:
                            tot_pos[i][j] += len(sc2_1) + len(sc2_2)

        
        faction_ave_pos = 0
        tot_pos_f = 0
        for pos in range(num_players):
            if pos == 0:
                faction_ave_pos += len(factions_pos[pos])
            tot_pos_f += len(factions_pos[pos])
        faction_ave_pos /= tot_pos_f
        
        for i in range(len(scores)):
            for j in range(len(scores[i])):
                if scores[i][j] is not None:
                    scores[i][j] /= num_players
                    scores[i][j] /= 2 
                    scores_overall[i][j] /= num_players
                    scores_overall[i][j] /= 2
                    
                    scores_diff[i][j] = scores[i][j] - scores_overall[i][j]
                    scores_overall[i][j] = round(scores_overall[i][j], 2)
                    
                    ave_pos[i][j] /= tot_pos[i][j]
                    ave_pos_min_faction[i][j] = ave_pos[i][j] - faction_ave_pos
                    ave_pos_min_faction[i][j] *= 100
                    ave_pos[i][j] *= 100
        
        score_map = go.Heatmap(
            z = scores_diff,
            x = final_scorings,
            y = final_scorings,
            # zmin = 9,
            zmid = 0,
            # zmax = 27,
            colorscale = 'RdBu',
            colorbar_x=0.45,
            text = scores
        )
        
        """
        pos_map = go.Heatmap(
            z = scores,
            x = final_scorings,
            y = final_scorings,
            # zmin = 9,
            # zmid = 18,
            # zmax = 27,
            colorscale = 'Greens'
        )
        """
        pos_map = go.Heatmap(
            z = ave_pos_min_faction,
            x = final_scorings,
            y = final_scorings, 
            # zmin = 9,
            zmid = 0,
            # zmax = 27,
            colorscale = 'RdBu',
            text=ave_pos
        )
        
    
        
        fig = make_subplots(rows=1, cols=2, horizontal_spacing=0.15, subplot_titles=("scoring drifference from all-faction average",  "\u0394 in victory probability (" + str(round(faction_ave_pos * 100, 2)) + "%)"), shared_yaxes=True)
        
        fig.append_trace(score_map, row=1, col=1)
        fig.append_trace(pos_map, row=1, col=2)
        # fig.update_yaxes(visible=False)
        fig.update_layout(title_text="Final scoring combination preferences for : " + faction)
        fig.show()

In [10]:
make_final_scoring_heatmap(
    dat_4,
    4)

/tmp/ipykernel_171486/1098605690.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

